In [1]:
# -*- coding: utf-8 -*-
import warnings
import os.path
import sys
import pickle
import datetime
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
import ow_f01, cxmn_train, cxmn_predict, predict_mean
from code.refactor.common import loadSettingsFromYamlFile, save_object, object2Float, get_column_by_type

warnings.filterwarnings("ignore", category=DeprecationWarning)

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
pd.options.display.max_columns=999
pd.options.display.max_rows=999

In [3]:
p1_used_header = ['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']
p2_used_header = ['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
				         'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
				         'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
				         'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
				         'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']
suffix = '.da'
item = 'p1'
for_what = ['train', 'predict']

In [4]:
def statitics_mape(new_df_final):
		new_df_sku = new_df_final.groupby('ProductKey').sum().reset_index()
		print "ensemble pred sum : %f"%(new_df_sku.ypred_mean_promo_new.sum())
		print "raw pred sum :      %f"%(new_df_sku.ypred_raw.sum())
		print "actual sum:         %f"%(new_df_sku.salesForecast.sum())

		print "raw pred residual:      %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)))
		print "ensemble pred residual: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)))

		print "raw pred mape: %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())
		print "ensemble mape: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())

In [5]:
def get_online_history_data():
    # p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_p1.da',sep='\t',header=None)
    # p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_p2.da',sep='\t',header=None)
    # ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/shishang/12029/12029_ts.da',sep='\t',header=None)
    p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_p1.da', sep='\t', header=None)
    p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_p2.da', sep='\t', header=None)
    ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/7054_ts.da', sep='\t', header=None)
    #p1 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/736_p1.da', sep='\t', header=None)
    #p2 = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/736_p2.da', sep='\t', header=None)
    #ts = pd.read_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/736_ts.da', sep='\t', header=None)
    # p1 = pd.read_csv('./diannaobangong/700_p1.csv', sep='\t', header=None)
    # p2 = pd.read_csv('./diannaobangong/700_p2.csv', sep='\t', header=None)
    # ts = pd.read_csv('./diannaobangong/700_ts.csv', sep='\t', header=None)
    # p1 = pd.read_csv('./diannaobangong/7371_p1.csv', sep='\t', header=None)
    # p2 = pd.read_csv('./diannaobangong/7371_p2.csv', sep='\t', header=None)
    # ts = pd.read_csv('./diannaobangong/7371_ts.csv', sep='\t', header=None)
    
    return p1,p2,ts


In [6]:
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)

In [7]:
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
seasonality_df = pd.read_csv('tmp/data/870_season.csv', parse_dates=['Date'])

In [8]:

#cate = 736 # 7371 # 700 # cate = 736
#valid_date = pd.to_datetime('2018-04-10')
#pred_date = pd.to_datetime('2018-05-08')
#scenario['lookforwardPeriodDays'] = 28

cate = 7054
valid_date = pd.to_datetime('2018-01-04')
pred_date = pd.to_datetime('2018-02-01')
scenario['lookforwardPeriodDays'] = 28

# cate = 12029
# pred_date = pd.to_datetime('2018-05-07')
# scenario['lookforwardPeriodDays'] = 7


In [9]:
p1_df,p2_df,ts_df = get_online_history_data()

In [10]:
p1_df.columns = p1_used_header
p2_df.columns = p2_used_header


In [11]:
###handle p2
p2_df['Date'] = pd.to_datetime(p2_df['Date'])
p2_df['dt'] = pd.to_datetime(p2_df['dt'])
p2_df.replace('null', np.nan, inplace=True)
p2_df.replace('None', np.nan, inplace=True)
p2_df.replace(-999, np.nan, inplace=True)
p2_df.drop_duplicates(inplace=True)

# Convert Object -> float
if 'object' in p2_df.dtypes.values:
    obj_cols = get_column_by_type(p2_df,'object')
    object2Float(p2_df,obj_cols)


convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag


In [12]:
def handle_f01(p1_df, for_what, area_rdc_map, pred_date, scenario):
    for fw in for_what:
    	print "output and save: %s_p1_%s"%(str(cate),fw)
    	train_pred_gate = fw   # 'train'
    	f01_output = ow_f01.generate_f01_promo(area_rdc_map, p1_df, scenario, train_pred_gate, ForecastStartDate=pred_date)
    	if train_pred_gate == 'train':
    		train_p1_df = f01_output
    	else:
    		predict_p1_df = f01_output
    return train_p1_df, predict_p1_df

train_p1_df,predict_p1_df = handle_f01(p1_df, for_what, area_rdc_map, pred_date, scenario)

output and save: 7054_p1_train
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert to Float columns:
bundle_subtype4_giveaway3
co

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
code/refactor/common.py:2111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  period_promo_raw['rdc_list'] = period_promo_raw['areatypearray'].apply(lambda x: extract_rdc(x, area_rdc_dict))
/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.

('| - Before filter: ', 51610)
('| - Filtered tokens: ', 48511)
('| - Filtered descriptions: ', 48511)
('| - Filtered flash sale available units', 47446)
('| - Filtered non-national promo: ', 45238)
| | - Add additional columns
| | - Calculate dd price per promotion
| | - Calculate dd price per period
| | - Generate bundle features
| | - Calculate bundle price per period
| - Reformat date
| | - process features
| | - aggregate features per period
| | - prep data for Step 3
| | - fix indicators on days with end time 00:00 or start time 23:59
| | - calculate weighted price per day
| | - aggregate features per day
f01 shape
(76155, 23)
output and save: 7054_p1_predict
('| - Before filter: ', 6016)
('| - Filtered tokens: ', 5658)
('| - Filtered descriptions: ', 5607)
('| - Filtered flash sale available units', 5591)
('| - Filtered non-national promo: ', 5576)
| | - Add additional columns
| | - Calculate dd price per promotion
| | - Calculate dd price per period
| | - Generate bundle featur

In [17]:
train_p1_df.to_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/p1_train_7054.csv',index=False)
predict_p1_df.to_csv('/home/ubuntu/sunjiadong/promotion_offline/tmp/data/p1_predict_7054.csv',index=False)

In [13]:
print train_p1_df.shape
print predict_p1_df.shape

(76155, 23)
(22691, 23)


In [14]:
ts_df.columns = ['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
ts_df['Date'] = pd.to_datetime(ts_df['Date'])

ts_df.replace('null', np.nan, inplace=True)
ts_df.replace(-999, np.nan, inplace=True)
ts_df.replace('None', np.nan, inplace=True)
ts_to_float_col = ['RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
if 'object' in ts_df[ts_to_float_col].dtypes.values:
    object2Float(ts_df,ts_to_float_col)


levels = ['HierarchyLevel3Key','ProductKey','RDCKey','Date']
ts_df.sort_values(levels, ascending=[True]*len(levels), inplace=True)
ts_df= ts_df.reset_index(drop=True)

In [15]:
ts_df_bak = ts_df.copy()

In [16]:
print ts_df.Date.min()
print ts_df.Date.max()

2015-04-18 00:00:00
2018-07-28 00:00:00


In [17]:
ts_df.head()

,Date,ind,RDCKey,ProductKey,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,brand_code,sales,priceAfterDiscount,jd_prc,vendibility,counterState,salesForecast,reserveState,stockQuantity,utc_flag
0,2015-04-18,1197,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2015-04-19,1196,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2015-04-20,1195,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2015-04-21,1194,3,117064,670,729,736,8740,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,2015-04-22,1193,3,117064,670,729,736,8740,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [18]:
ts_df[['RDCKey','ProductKey']].drop_duplicates().shape

(9789, 2)

In [19]:
# -*- coding: utf-8 -*-
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling,calculateRolling,calculateLagging,createDateFeatures,splitTimeWindow,calculateSimilarRolling,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [20]:
ForecastStartDate = pd.to_datetime(pred_date)
print scenario['lookbackPeriodDays']

400


In [21]:
print ForecastStartDate

2018-05-08 00:00:00


In [22]:
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])

In [23]:
print DataStartDate

2017-04-03 00:00:00


In [24]:
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
object2Int(holidays_df,['Holiday','Ind_1111_pre','Ind_1111','Ind_1111_post','Ind_618_pre','Ind_618','Ind_618_post','Ind_1212'])
for col in area_rdc_map.columns:
    area_rdc_map[col] = area_rdc_map[col].astype('int')
period_promo_raw = train_p1_df
promoCalendarDf = p2_df

convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212


In [25]:
promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']<ForecastStartDate)&(promoCalendarDf['Date']>DataStartDate)]

ts_df_pred = ts_df[(ts_df['Date']<ForecastStartDate)&(ts_df['Date']>DataStartDate)]


period_promo_raw.drop_duplicates(inplace=True)
promo_period = period_promo_raw


In [26]:
raw_df= ts_df_pred[ts_df_pred['Date']<ForecastStartDate]


''' initialization'''
if 'ind' in raw_df.columns.values:
    del raw_df['ind']
groupKeys = scenario['selectedColumns']['levelsCoarse'] # ['HierarchyLevel3Key','ProductKey','RDCKey']

In [27]:
min_dt_record = raw_df.groupby(groupKeys)['Date'].min().reset_index(name='dt_min')
min_dt_onstock_record = raw_df[raw_df.counterState==1].groupby(groupKeys)['Date'].min().reset_index(name='dt_onstock_min')
merged_min_dt = pd.merge(min_dt_record,min_dt_onstock_record,on=groupKeys)
valid_merged_min_dt = merged_min_dt[merged_min_dt.dt_min ==merged_min_dt.dt_onstock_min]

raw_df = pd.merge(raw_df, valid_merged_min_dt[groupKeys+['dt_onstock_min']],how='left',on=groupKeys)
raw_df.reset_index(drop=True,inplace=True)
''' fillZeroRows '''
raw_df= filter_non_price_fill_it(raw_df, scenario, price_col_ts='priceAfterDiscount')

In [28]:
q = scenario['quantile'] # 0.95

unitVolumeQuantile = raw_df.groupby(groupKeys)[scenario['Target_col']].quantile(q, interpolation='lower')
quantileDf = pd.DataFrame(unitVolumeQuantile).reset_index()
quantileDf.rename(columns = {scenario['Target_col']: 'UnitCapVolume'}, inplace=True)

# join quantile
raw_df= pd.merge(raw_df, quantileDf, how='left', on=groupKeys)
raw_df['OrderNonOutlierVolume'] = raw_df[['salesForecast', 'UnitCapVolume']].min(axis=1)

# drop cap volume
raw_df.drop(['UnitCapVolume'], axis=1, inplace = True)
if True:
    f = lambda x: x.shift(1).fillna(method='bfill')
    raw_df[['counterState1']] = raw_df[['counterState','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)
    raw_df[['vendibility1']] = raw_df[['vendibility','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)

featuresDf = raw_df


In [29]:
featuresDf.Date.max()

Timestamp('2018-05-07 00:00:00')

In [30]:
if seasonality_df is None:
    if (ts_df_pred.Date.max() - ts_df_pred.Date.min()).days > 730:
        print 'running Seasonality'
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key', 'ProductKey'], 'sku', scenario)
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key'], 'level3', scenario)

        '''
            createLevel3Features
        '''
        featuresDf = createLevel3Features(featuresDf,scenario)
    else:
        print 'faking Seasonality Due to less 2 year Data'
        featuresDf['skuDecomposedTrend'],featuresDf['skuDecomposedSeasonal'],featuresDf['level3DecomposedTrend'],featuresDf['level3DecomposedSeasonal'],featuresDf['Curve'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
else:
    if 'OrderNonOutlierVolume' in seasonality_df.columns:
        seasonality_df.drop(['OrderNonOutlierVolume'], axis=1, inplace = True)
    featuresDf = pd.merge(featuresDf,seasonality_df,on=['HierarchyLevel3Key','ProductKey','Date','RDCKey'],how='left')
'''加入Holiday '''

'\xe5\x8a\xa0\xe5\x85\xa5Holiday '

In [31]:
featuresDf = pd.merge(featuresDf, holidays_df, on='Date', how='left')
'''
    createDateFeatures
'''
featuresDf = createDateFeatures(featuresDf)

'''
    rolling part, 28min, too slow!
'''

featuresDf = pd.merge(featuresDf, promoCalendarDf[promoCalendarDf.columns.difference(['HierarchyLevel1Key','HierarchyLevel2Key', 'HierarchyLevel3Key'])],   left_on = ['ProductKey', 'Date'], right_on = ['ProductKey', 'Date'], how='left')
if scenario['STOCK_PRICE_FLAG']:
    featuresDf = calculateStockFeatures(featuresDf)


featuresDf = calculateNationalRolling(featuresDf, scenario, scenario['selectedColumns']['target'])
featuresDf = calculateRolling(featuresDf, scenario, scenario['selectedColumns']['target'])
featuresDf = calculateSimilarRolling(featuresDf, scenario, scenario['Target_col'])
featuresDf = calculateLagging(featuresDf, scenario, scenario['Target_col'])

featuresDf = pd.merge(featuresDf,  promo_period,on = ['ProductKey', 'Date'], how = 'left')

| Add date features...
| - Add month of the year...
| - Add day of the week...
| Calculating national rolling features...
('| - Rolling value:', 'xxxHashColumn')
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 28 mean...
| - Rolling decay 14 mean...
| - Rolling decay 7 mean...
| - Rolling decay 3 mean...
| - Added rolling features in 72.670343 seconds 

| Calculating  rolling features...
('| - Rolling value:', 'OrderNonOutlierVolume')
| - Rolling 360 mean...
| - Rolling 180 mean...
| - Rolling 90 mean...
| - Rolling 28 mean...
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 360 mean...
| - Rolling decay 180 mean...
| - Rolling decay 90 mean...
| - Rolling decay 28 mean...
| - Rolling dec

In [32]:
featuresDf.shape

(2319750, 141)

In [34]:
list(missing_data.index)

NameError: name 'missing_data' is not defined

In [35]:
promo_exclude_list = ['bd_discount_sgp_wgt',
 'suit_ind',
 'bd_discount_wgt',
 'Curve',
 'level3DecomposedSeasonal',
 'level3DecomposedTrend',
 'skuDecomposedTrend',
 'skuDecomposedSeasonal',
 'dd_price_weighted',
 'bd_price_weighted',
 'dd_ind',
 'bundle_ind',
 'bundle_buy199get100_ind',
 'flashsale_ind',
 'freegift_ind',
 'dd_price_daily_min',
 'dd_discount_sgp_wgt',
 'cnt_period',
 'sgp_daily_min',
 'jdprice_daily_min',
 'bd_price_daily_min',
 'dd_discount_wgt',
 'suit_discount_new_daily_max',
 'bd_discount_daily_max',
 'dd_discount_sgp_daily_max',
 'dd_discount_daily_max',
 'syntheticgrossprice_vb',
 'bundleflag',
 'daynumberinpromotion',
 'directdiscountflag',
 'durationinhours',
 'freegiftflag',
 'jdprice',
 'promotionkey',
 'promotionsubtype',
 'promotiontype',
 'syntheticdiscounta_vb',
 'suitflag',
 'MaxSyntheticDiscountA',
 'MinSyntheticDiscountA',
 'SyntheticGrossPrice',
 'AvgSyntheticDiscountA',
 'PromotionCount',
 'dt',
 'MinBundleDiscount',
 'MinDirectDiscount',
 'MinDiscount',
 'MinFreegiftDiscount',
 'MaxDirectDiscount',
 'bundlecount',
 'MaxBundleDiscount',
 'AvgFreegiftDiscount',
 'AvgDiscount',
 'MaxDiscount',
 'AvgDirectDiscount',
 'AvgBundleDiscount',
 'numberhierarchylevel1',
 'numberhierarchylevel2',
 'numberhierarchylevel3',
 'numberproducts',
 'stockprice',
 'strongmark',
 'MaxFreegiftDiscount']

In [33]:
xColumns = scenario['selectedColumns']['features']
yColumns = scenario['selectedColumns']['target']
xColumns.remove('skuDecomposedTrend')
xColumns.remove('skuDecomposedSeasonal')
xColumns.remove('level3DecomposedTrend')
xColumns.remove('level3DecomposedSeasonal')
xColumns.remove('Curve')

In [97]:
exclude_cols = ['MaxBundleDiscount',
 'numberproducts',
 'bd_price_daily_min',
 'MinBundleDiscount',
 'AvgBundleDiscount',
 'MaxFreegiftDiscount',
 'MinFreegiftDiscount',
 'AvgFreegiftDiscount',
 'strongmark',
 'bundle_buy199get100_ind',
 'suit_ind',
 'freegift_ind',
 'suit_discount_new_daily_max']
for col in exclude_cols:
    xColumns.remove(col)

In [98]:
for col in promo_exclude_list:
    if col in xColumns:
        xColumns.remove(col)

In [34]:
if 'RDCKey' not in xColumns:
    print "go"
    xColumns.append('RDCKey')

In [35]:
import copy
model_1_xColumns = copy.deepcopy(xColumns)
model_1_xColumns.remove('RDCKey')

In [36]:
featuresDf.RDCKey = featuresDf.RDCKey.astype('float')

In [37]:
def train_model(scenario, df_feature):
    param = {
    'objective': "reg:linear",
    'booster': "gbtree",
    'eta' : 0.3, # 0.06, #0.01,
    'min_child_weight':1.0,
    'gamma':0,
    'seed':10,
    'max_depth' :10, #changed from default of 8
    'subsample' :1.0, # 0.7
    'colsample_bytree': 1.0,# 0.7
    # 'silent':1,
    'nthread':4,
    'num_round':  20
    }

    y = df_feature[yColumns].values
    
    X = df_feature[model_1_xColumns]
    xgmat = xgb.DMatrix(X.values, label=y, missing=np.NaN)
    model = xgb.train(param, xgmat, int(param['num_round']))
    return model

In [38]:
model_hash = {}
grouped = featuresDf.groupby('RDCKey')
for rdc, group in grouped:
    print 'rdc'
    print rdc
    this_model = train_model(scenario, group)
    model_hash[rdc] = this_model

rdc
3.0
rdc
4.0
rdc
5.0
rdc
6.0
rdc
9.0
rdc
10.0
rdc
316.0
rdc
772.0


In [39]:
print model_hash[3].get_fscore()

def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1
    outfile.close()

features = [x for x in featuresDf[model_1_xColumns].columns]
ceate_feature_map(features)

importance = model_hash[3].get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items())
df = pd.DataFrame(importance, columns=['feature', 'fscore'])

df.sort_values(by='fscore', ascending=False)

{'rolling14Mean': 132, 'volume_rolling180Shift': 164, 'rolling360Mean': 117, 'similar_rolling7DecayMean': 59, 'bd_discount_wgt': 44, 'national_rolling7Mean': 98, 'similar_rolling7Mean': 103, 'similar_rolling14Mean': 127, 'MinDiscount': 89, 'national_rolling14Median': 96, 'MinDirectDiscount': 18, 'numberproducts': 44, 'national_rolling2Mean': 132, 'strongmark': 1, 'rolling14Median': 89, 'similar_rolling2Mean': 92, 'sgp_daily_min': 28, 'AvgDiscount': 31, 'rolling5Mean': 149, 'MaxSyntheticDiscountA': 122, 'rolling180Mean': 261, 'rolling7Mean': 174, 'DayOfWeek': 424, 'jdprice_daily_min': 24, 'AvgBundleDiscount': 2, 'rolling1Mean': 197, 'rolling7Median': 90, 'Ind_1111_pre': 18, 'cnt_period': 28, 'similar_rolling3DecayMean': 50, 'bundle_ind': 15, 'national_rolling1Mean': 271, 'rolling28Mean': 241, 'Month': 315, 'flashsale_ind': 20, 'volume_rolling7Shift': 187, 'MaxDirectDiscount': 42, 'MaxBundleDiscount': 44, 'rolling90DecayMean': 101, 'rolling2Mean': 140, 'national_rolling3Mean': 113, 'dd_d

In [ ]:
['MaxBundleDiscount',
 'numberproducts',
 'bd_price_daily_min',
 'MinBundleDiscount',
 'AvgBundleDiscount',
 'MaxFreegiftDiscount',
 'MinFreegiftDiscount',
 'AvgFreegiftDiscount',
 'strongmark',
 'bundle_buy199get100_ind',
 'suit_ind',
 'freegift_ind',
 'suit_discount_new_daily_max']

In [42]:
scenario['lookforwardPeriodDays']

28

In [39]:
PredictEndDate = ForecastStartDate + datetime.timedelta(days=(scenario['lookforwardPeriodDays']-1))
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays_predict'])


print PredictEndDate
print scenario['lookbackPeriodDays_predict']

ts_df = ts_df[(ts_df['Date']<ForecastStartDate)&(ts_df['Date']>DataStartDate)]

holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
object2Int(holidays_df,['Holiday','Ind_1111_pre','Ind_1111','Ind_1111_post','Ind_618_pre','Ind_618','Ind_618_post','Ind_1212'])
for col in area_rdc_map.columns:
    area_rdc_map[col] = area_rdc_map[col].astype('int')
period_promo_raw = predict_p1_df
promoCalendarDf = p2_df

promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']<PredictEndDate)&(promoCalendarDf['Date']>DataStartDate)]


period_promo_raw.drop_duplicates(inplace=True)
promo_period = period_promo_raw


''' f02 predict'''

raw_df= ts_df[ts_df['Date']<ForecastStartDate]

if 'ind' in raw_df.columns.values:
    del raw_df['ind']
groupKeys = scenario['selectedColumns']['levelsCoarse'] # ['HierarchyLevel3Key','ProductKey','RDCKey']

''' 获取新品的上柜时间 'dt_onstaock_min'. nan if 上柜时间小于ts start_time '''
min_dt_record = raw_df.groupby(groupKeys)['Date'].min().reset_index(name='dt_min')
min_dt_onstock_record = raw_df[raw_df.counterState==1].groupby(groupKeys)['Date'].min().reset_index(name='dt_onstock_min')
merged_min_dt = pd.merge(min_dt_record,min_dt_onstock_record,on=groupKeys)
valid_merged_min_dt = merged_min_dt[merged_min_dt.dt_min ==merged_min_dt.dt_onstock_min]

raw_df = pd.merge(raw_df, valid_merged_min_dt[groupKeys+['dt_onstock_min']],how='left',on=groupKeys)
raw_df.reset_index(drop=True,inplace=True)
''' fillZeroRows '''
raw_df= filter_non_price_fill_it(raw_df, scenario, price_col_ts='priceAfterDiscount')

q = scenario['quantile'] # 0.95
# q = 0.9

unitVolumeQuantile = raw_df.groupby(groupKeys)[scenario['Target_col']].quantile(q, interpolation='lower')
quantileDf = pd.DataFrame(unitVolumeQuantile).reset_index()
quantileDf.rename(columns = {scenario['Target_col']: 'UnitCapVolume'}, inplace=True)

# join quantile
raw_df= pd.merge(raw_df, quantileDf, how='left', on=groupKeys)
raw_df['OrderNonOutlierVolume'] = raw_df[['salesForecast', 'UnitCapVolume']].min(axis=1)

# drop cap volume
raw_df.drop(['UnitCapVolume'], axis=1, inplace = True)
if True:
    f = lambda x: x.shift(1).fillna(method='bfill')
    raw_df[['counterState1']] = raw_df[['counterState','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)
    raw_df[['vendibility1']] = raw_df[['vendibility','ProductKey','RDCKey']].groupby(['ProductKey','RDCKey']).transform(f)

df_past = raw_df

def return_nearest_date_index():
    index = [i for i in range(1,10) if (ForecastStartDate- pd.DateOffset(days=i)).strftime('%Y-%m-%d') not in scenario['Remove_by_stat_date']]
    return index[0]

fut_train_end = ForecastStartDate- pd.DateOffset(days=1)
# make sure train end date not in remove date
if fut_train_end.strftime('%Y-%m-%d') in scenario['Remove_by_stat_date']:
    i = return_nearest_date_index()

    fut_train_end = ForecastStartDate- pd.DateOffset(days=i)

''' concat Future Df by RDC'''
dfs=[]
for rdc in list(set(df_past.RDCKey)):

    valid_skus_fut=list(set(df_past[(df_past['Date']==fut_train_end)&(df_past.RDCKey==rdc)]['ProductKey'].values))

    # generate_default_df
    def generate_strutured_null_df(data_start_date, data_end_date, valid_skus, sku_name='ProductKey', date_name='Date'):
        rng = pd.date_range(start=data_start_date, end=data_end_date, freq='D')
        valid_date_span_length = len(rng)
        sku_col = []
        rng_col = []
        for sku_id in valid_skus:
            sku_col.extend(np.repeat(sku_id, valid_date_span_length))
            rng_col.extend(rng)

        dic = {sku_name : sku_col, date_name: rng_col}

        return pd.DataFrame(dic)

    fut_null_df = generate_strutured_null_df(ForecastStartDate, PredictEndDate, valid_skus_fut)  # Filling 0601-0604 cols [stat_date, sku]
    fill_one_cols = {'vendibility1': 1}

    basic_col=['ProductKey','Date','RDCKey','HierarchyLevel1Key','HierarchyLevel2Key','HierarchyLevel3Key'\
            ,'dt_onstock_min','brand_code','salesForecast','reserveState','OrderNonOutlierVolume','vendibility1']
    to_fillna_col=['dt_onstock_min','RDCKey','brand_code','reserveState','HierarchyLevel1Key','HierarchyLevel2Key'\
            ,'HierarchyLevel3Key','vendibility1']

    default_kv = generate_default_values_by_dic(fill_nan_list=to_fillna_col, fill_default_kv=fill_one_cols)

    fut_null_df = fill_col_with_default(fut_null_df, default_kv)

    df_future = df_past[df_past.RDCKey==rdc][basic_col].append(fut_null_df)
    df_future.sort_values(['ProductKey', 'Date'], inplace=True)
    df_future.reset_index(drop=True,inplace=True)
    col = list(df_future.columns.values)
    col.remove('ProductKey')

    df_future[col] = df_future.groupby(['ProductKey']).transform(lambda x: x.fillna(method='ffill'))
    dfs.append(df_future)
df_with_future=pd.concat(dfs)
df_with_future.reset_index(drop=True,inplace=True)

featuresDf = df_with_future

if seasonality_df is None:
    if (ts_df.Date.max() - ts_df.Date.min()).days > 730:
        print 'running Seasonality'
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key', 'ProductKey'], 'sku', scenario)
        featuresDf = createSeasonalityDecomposeFeatures(featuresDf, ['HierarchyLevel3Key'], 'level3', scenario)

        '''
            createLevel3Features
        '''
        featuresDf = createLevel3Features(featuresDf,scenario)
    else:
        print 'faking Seasonality Due to less 2 year Data'
        featuresDf['skuDecomposedTrend'],featuresDf['skuDecomposedSeasonal'],featuresDf['level3DecomposedTrend'],featuresDf['level3DecomposedSeasonal'],featuresDf['Curve'] = [np.nan,np.nan,np.nan,np.nan,np.nan]
else:
    if 'OrderNonOutlierVolume' in seasonality_df.columns:
        seasonality_df.drop(['OrderNonOutlierVolume'], axis=1, inplace = True)
    featuresDf = pd.merge(featuresDf,seasonality_df,on=['HierarchyLevel3Key','ProductKey','Date','RDCKey'],how='left')

# -*- coding: utf-8 -*-
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling_predict,calculateRolling_predict,calculateLagging_predict,createDateFeatures,splitTimeWindow,calculateSimilarRolling_predict,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
import pickle

featuresDf = pd.merge(featuresDf, holidays_df, on='Date', how='left')

'''
    createDateFeatures
'''
featuresDf = createDateFeatures(featuresDf)

'''
    rolling part, 28min, too slow!
'''

# featuresDf = pd.merge(featuresDf, promoCalendarDf[promoCalendarDf.columns.difference(['HierarchyLevel1Key','HierarchyLevel2Key', 'HierarchyLevel3Key'])],   left_on = ['ProductKey', 'Date'], right_on = ['ProductKey', 'Date'], how='left')
featuresDf = pd.merge(featuresDf, promoCalendarDf[promoCalendarDf.columns.difference(['HierarchyLevel1Key','HierarchyLevel2Key', 'HierarchyLevel3Key'])].drop_duplicates(),   on = ['ProductKey', 'Date'], how='left')
if scenario['STOCK_PRICE_FLAG']:
    featuresDf = calculateStockFeatures(featuresDf)

df_fut = featuresDf[featuresDf.Date>=pd.to_datetime(ForecastStartDate)]
# df_fut = featuresDf[featuresDf.Date>=pd.to_datetime('2017-10-01')]
df_past = featuresDf[featuresDf.Date<pd.to_datetime(ForecastStartDate)]
####notice: use selected promotion cols' mean values to fill the same cols in df_future

df_fut = calculateNationalRolling_predict(df_fut,df_past, scenario, scenario['selectedColumns']['target'])
df_fut = calculateRolling_predict(df_fut,df_past, scenario, scenario['selectedColumns']['target'])
df_fut = calculateSimilarRolling_predict(df_fut,df_past, scenario, scenario['Target_col'])
df_fut = calculateLagging_predict(df_fut,df_past, scenario, scenario['Target_col'])

# promo_period.read_csv('f01.csv',parse_dates=['Date'])
df_fut = pd.merge(df_fut,  promo_period,on = ['ProductKey', 'Date'], how = 'left')

2018-06-04 00:00:00
400
convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
| Add date features...
| - Add month of the year...
| - Add day of the week...


code/refactor/common.py:1363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  + df_past[similarLevels[2]]
code/refactor/common.py:1364: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_past['SyntheticPromotionSubType'] = df_past['SyntheticPromotionSubType'].fillna(0.0)


| - group by SyntheticPromotionSubType...


In [40]:
grouped = df_fut.groupby('RDCKey')
result_list = []
for rdc, pred_df in grouped:
    if rdc in model_hash.keys():
        this_model = model_hash[rdc]
    else:
        continue
    
    X_future = pred_df[model_1_xColumns]

    future_xtest = xgb.DMatrix(X_future.values, missing=np.NaN )
    ypred = this_model.predict(future_xtest)
    pred_df['ypred'] =ypred
    pred_df['RDCKey'] = rdc

    ''' Tuning result '''
    lanjie = pred_df[(pred_df.ypred<0)]
    if len(lanjie)>0:
        pred_df.ix[lanjie.index,'ypred'] = 0
    result_list.append(pred_df)
final_result = pd.concat(result_list)

/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [41]:
final_result.shape

(197176, 135)

In [42]:
def get_actual_sales(ts_df, pred_date, scenario):
	simplified_ts_df = ts_df[ts_df.Date.between(pred_date, pd.to_datetime(pred_date)+\
                       pd.DateOffset(days=scenario['lookforwardPeriodDays']-1))]
	return simplified_ts_df
simplified_ts_df = get_actual_sales(ts_df_bak, pred_date, scenario)


In [43]:
simplified_ts_df.shape

(255910, 17)

In [44]:
simplified_ts_df.head()

,Date,ind,RDCKey,ProductKey,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,brand_code,sales,priceAfterDiscount,jd_prc,vendibility,counterState,salesForecast,reserveState,stockQuantity,utc_flag
1116,2018-05-08,81,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1117,2018-05-09,80,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1118,2018-05-10,79,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1119,2018-05-11,78,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1120,2018-05-12,77,3,117064,670,729,736,8740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [45]:
simplified_ts_df.Date.max()

Timestamp('2018-06-04 00:00:00')

In [46]:
list_keys = ['Date','RDCKey','ProductKey']
df = pd.merge(final_result, simplified_ts_df[list_keys + ['sales','salesForecast']], how='left', on=list_keys)

In [47]:
df.shape

(197176, 137)

In [48]:
df.head()

,Date,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,OrderNonOutlierVolume,ProductKey,RDCKey,brand_code,dt_onstock_min,reserveState,salesForecast_x,vendibility1,skuDecomposedSeasonal,skuDecomposedTrend,level3DecomposedTrend,level3DecomposedSeasonal,Curve,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,AvgBundleDiscount,AvgDirectDiscount,AvgDiscount,AvgFreegiftDiscount,AvgSyntheticDiscountA,MaxBundleDiscount,MaxDirectDiscount,MaxDiscount,MaxFreegiftDiscount,MaxSyntheticDiscountA,MinBundleDiscount,MinDirectDiscount,MinDiscount,MinFreegiftDiscount,MinSyntheticDiscountA,PromotionCount,SyntheticGrossPrice,bundlecount,bundleflag,daynumberinpromotion,directdiscountflag,dt,durationinhours,freegiftflag,jdprice,numberhierarchylevel1,numberhierarchylevel2,numberhierarchylevel3,numberproducts,promotionkey,promotionsubtype,promotiontype,stockprice,strongmark,suitflag,syntheticdiscounta_vb,syntheticgrossprice_vb,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,SyntheticPromotionSubType,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,suit_ind,freegift_ind,cnt_period,dd_discount_daily_max,dd_discount_sgp_daily_max,bd_discount_daily_max,suit_discount_new_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_wgt,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,ypred,sales,salesForecast_y
0,2018-05-08,670.0,729.0,736.0,462.0,127768,3,624.0,2017-04-04,0.0,525.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,0,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375.142857,344.142857,372.8,355.666667,499.0,839.0,389.0,275.0,357.290343,361.335157,394.959097,518.069938,167.919444,168.744444,121.022222,132.142857,140.785714,125.142857,144.8,181.333333,255.5,462.0,133.0,49.0,161.676545,154.076934,143.745778,139.508878,143.677748,171.169049,260.498202,0.0,145.285714,134.142857,157.4,202.333333,287.0,525.0,133.0,49.0,143.729905,152.171873,186.922884,291.998202,310.0,89.0,199.0,458.0,46.0,171.0,171.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,171.0,171.0,171.0,171.0,0.0,0.0,0.0,0.0,386.302582,304.0,304.0
1,2018-05-09,670.0,729.0,736.0,462.0,127768,3,624.0,2017-04-04,0.0,525.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,0,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375.142857,344.142857,372.8,355.666667,499.0,839.0,389.0,275.0,357.290343,361.335157,394.959097,518.069938,167.919444,168.744444,121.022222,132.142857,140.785714,125.142857,144.8,181.333333,255.5,462.0,133.0,49.0,161.676545,154.076934,143.745778,139.508878,143.677748,171.169049,260.498202,0.0,145.285714,134.142857,157.4,202.333333,287.0,525.0,133.0,49.0,143.729905,152.171873,186.922884,291.998202,310.0,89.0,199.0,458.0,46.0,171.0,171.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,171.0,171.0,171.0,171.0,0.0,0.0,0.0,0.0,386.302582,217.0,217

In [49]:
df.Date.max()

Timestamp('2018-06-04 00:00:00')

In [50]:
df.to_csv('./result_670/raw_pred_result_736_0508-step28.csv',index=False)
# df.to_csv('./result_670/raw_pred_result_7371_0508-step28.csv',index=False)

In [51]:
print df.ypred.sum()
print df.salesForecast_y.sum()

1315781.0
1357072.0


In [116]:
df.shape

(601468, 137)

In [51]:
df2 = df.groupby(['ProductKey','RDCKey'])['ypred','salesForecast_y'].sum().reset_index()
print np.sum(np.abs(df2.ypred - df2.salesForecast_y)) / np.sum(df2.salesForecast_y)

0.3231337373910933


In [59]:
df1 = df[df.Date<='2018-06-04']
df2 = df1.groupby(['ProductKey','RDCKey'])['ypred','salesForecast_y'].sum().reset_index()
print np.sum(np.abs(df2.ypred - df2.salesForecast_y)) / np.sum(df2.salesForecast_y)

0.3231337373910933


In [54]:
featuresDf.head()

,Date,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,OrderNonOutlierVolume,ProductKey,RDCKey,brand_code,dt_onstock_min,reserveState,salesForecast,vendibility1,skuDecomposedSeasonal,skuDecomposedTrend,level3DecomposedTrend,level3DecomposedSeasonal,Curve,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,AvgBundleDiscount,AvgDirectDiscount,AvgDiscount,AvgFreegiftDiscount,AvgSyntheticDiscountA,MaxBundleDiscount,MaxDirectDiscount,MaxDiscount,MaxFreegiftDiscount,MaxSyntheticDiscountA,MinBundleDiscount,MinDirectDiscount,MinDiscount,MinFreegiftDiscount,MinSyntheticDiscountA,PromotionCount,SyntheticGrossPrice,bundlecount,bundleflag,daynumberinpromotion,directdiscountflag,dt,durationinhours,freegiftflag,jdprice,numberhierarchylevel1,numberhierarchylevel2,numberhierarchylevel3,numberproducts,promotionkey,promotionsubtype,promotiontype,stockprice,strongmark,suitflag,syntheticdiscounta_vb,syntheticgrossprice_vb
0,2017-04-04,670.0,729.0,7371.0,0.0,247046,3.0,15097.0,2017-04-04,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,3,0,0,0,0,0,0,0,4,1,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,1.0,30.0,1.0,1.0,9.0,0.0,2017-04-04,23.0,0.0,38.0,1.0,1.0,1.0,310.0,197408516.0,2.0,10.0,22.28,0.0,0.0,0.2425,30.0
1,2017-04-05,670.0,729.0,7371.0,1.0,247046,3.0,15097.0,2017-04-04,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,0,4,2,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,1.0,30.0,1.0,1.0,1.0,0.0,2017-04-05,24.0,0.0,38.0,1.0,1.0,1.0,310.0,197528824.0,2.0,10.0,22.27,0.0,0.0,0.2425,30.0
2,2017-04-06,670.0,729.0,7371.0,1.0,247046,3.0,15097.0,2017-04-04,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,1,0,0,0,4,3,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,1.0,30.0,1.0,1.0,2.0,0.0,2017-04-06,24.0,0.0,38.0,1.0,1.0,1.0,310.0,197528824.0,2.0,10.0,22.28,0.0,0.0,0.2425,30.0
3,2017-04-07,670.0,729.0,7371.0,3.0,247046,3.0,15097.0,2017-04-04,0.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,0,4,4,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,1.0,30.0,1.0,1.0,3.0,0.0,2017-04-07,24.0,0.0,38.0,1.0,1.0,1.0,310.0,197528824.0,2.0,10.0,22.28,0.0,0.0,0.2425,30.0
4,2017-04-08,670.0,729.0,7371.0,0.0,247046,3.0,15097.0,2017-04-04,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,2,0,0,0,0,0,0,0,4,5,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,0.40201,0.0,0.40201,0.0,0.2425,1.0,30.0,1.0,1.0,4.0,0.0,2017-04-08,24.0,0.0,38.0,1.0,1.0,1.0,310.0,197528824.0,2.0,10.0,22.28,0.0,0.0,0.2425,30.0


In [55]:
raw_df = featuresDf.copy()
raw_df_na = (raw_df.isnull().sum() / len(raw_df)) * 100

raw_df_na = raw_df_na.drop(raw_df_na[raw_df_na == 0].index).sort_values(ascending=False)

missing_data = pd.DataFrame({'Missing Ratio' :raw_df_na})

missing_data

,Missing Ratio
Curve,100.000000
skuDecomposedSeasonal,100.000000
skuDecomposedTrend,100.000000
level3DecomposedTrend,100.000000
level3DecomposedSeasonal,100.000000
jdprice,62.633710
bundleflag,62.633710
daynumberinpromotion,62.633710
directdiscountflag,62.633710
durationinhours,62.633710


In [128]:
exclude_cols

['MaxBundleDiscount',
 'numberproducts',
 'bd_price_daily_min',
 'MinBundleDiscount',
 'AvgBundleDiscount',
 'MaxFreegiftDiscount',
 'MinFreegiftDiscount',
 'AvgFreegiftDiscount',
 'strongmark',
 'bundle_buy199get100_ind',
 'suit_ind',
 'freegift_ind',
 'suit_discount_new_daily_max']

In [126]:
promo_exclude_list = list(missing_data.index)

In [127]:
raw_df = df.copy()
raw_df_na = (raw_df.isnull().sum() / len(raw_df)) * 100

raw_df_na = raw_df_na.drop(raw_df_na[raw_df_na == 0].index).sort_values(ascending=False)

missing_data = pd.DataFrame({'Missing Ratio' :raw_df_na})

missing_data

,Missing Ratio
skuDecomposedSeasonal,100.000000
skuDecomposedTrend,100.000000
level3DecomposedTrend,100.000000
level3DecomposedSeasonal,100.000000
Curve,100.000000
suitflag,55.862656
syntheticgrossprice_vb,55.862656
promotiontype,55.862656
promotionsubtype,55.862656
promotionkey,55.862656


In [61]:
print "lgm pred sum:           %s"%final_result.model_2_ypred.sum()
print "actual sales sum:       %s"%final_result.salesForecast.sum()
print "bottom up pred sum:     %s"%final_result.ypred.sum()
print "ensemble pred sum:      %s"%final_result.ensemble.sum()

final_sku_rdc = final_result.groupby(['RDCKey','ProductKey']).sum().reset_index()
final_sku = final_result.groupby(['ProductKey']).sum().reset_index()
print "\nbottom up pred mape:\n"

print "daily_rdc_sku mape :%s"%(np.sum(np.abs(final_result.ypred - final_result.salesForecast)) / np.sum(final_result.salesForecast))
print "28days_rdc_sku mape:%s"%(np.sum(np.abs(final_sku_rdc.ypred - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast))
print "28days_sku mape:    %s"%(np.sum(np.abs(final_sku.ypred - final_sku.salesForecast)) / np.sum(final_sku.salesForecast))
print "\nlgm pred mape:\n"
print "daily_rdc_sku mape  :%s"%(np.sum(np.abs(final_result.model_2_ypred - final_result.salesForecast)) / np.sum(final_result.salesForecast))
print "28days_rdc_sku mape :%s"%(np.sum(np.abs(final_sku_rdc.model_2_ypred - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast))
print "28days_sku mape     :%s"%(np.sum(np.abs(final_sku.model_2_ypred - final_sku.salesForecast)) / np.sum(final_sku.salesForecast))
print "\nensemble pred mape:\n"
print "daily_rdc_sku mape  :%s"%(np.sum(np.abs(final_result.ensemble - final_result.salesForecast)) / np.sum(final_result.salesForecast))
print "28days_rdc_sku mape :%s"%(np.sum(np.abs(final_sku_rdc.ensemble - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast))
print "28days_sku mape     :%s"%(np.sum(np.abs(final_sku.ensemble - final_sku.salesForecast)) / np.sum(final_sku.salesForecast))

AttributeError: 'DataFrame' object has no attribute 'model_2_ypred'

In [180]:
result2 = final_result[final_result.Date<='2018-05-14']

print result2.model_2_ypred.sum()
print result2.salesForecast.sum()
print result2.ypred.sum()
print result2.ensemble.sum()

final_sku_rdc = result2.groupby(['RDCKey','ProductKey']).sum().reset_index()
final_sku = result2.groupby(['ProductKey']).sum().reset_index()

print np.sum(np.abs(result2.ypred - result2.salesForecast)) / np.sum(result2.salesForecast)
print np.sum(np.abs(final_sku_rdc.ypred - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast)
print np.sum(np.abs(final_sku.ypred - final_sku.salesForecast)) / np.sum(final_sku.salesForecast)

print np.sum(np.abs(result2.model_2_ypred - result2.salesForecast)) / np.sum(result2.salesForecast)
print np.sum(np.abs(final_sku_rdc.model_2_ypred - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast)
print np.sum(np.abs(final_sku.model_2_ypred - final_sku.salesForecast)) / np.sum(final_sku.salesForecast)

print np.sum(np.abs(result2.ensemble - result2.salesForecast)) / np.sum(result2.salesForecast)
print np.sum(np.abs(final_sku_rdc.ensemble - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast)
print np.sum(np.abs(final_sku.ensemble - final_sku.salesForecast)) / np.sum(final_sku.salesForecast)

386979.61442904826
353955.0
312920.28
320854.73052179447
0.6221973253982767
0.401730627207566
0.28037285992501776
0.6192544957199012
0.3961479468583056
0.2540728292215213
0.5856877856905752
0.36357233691185026
0.24189731385176763


In [179]:
result3 = final_result[final_result.Date<='2018-05-21']

print result3.model_2_ypred.sum()
print result3.salesForecast.sum()
print result3.ypred.sum()
print result3.ensemble.sum()

final_sku_rdc = result3.groupby(['RDCKey','ProductKey']).sum().reset_index()
final_sku = result3.groupby(['ProductKey']).sum().reset_index()

print np.sum(np.abs(result3.ypred - result3.salesForecast)) / np.sum(result3.salesForecast)
print np.sum(np.abs(final_sku_rdc.ypred - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast)
print np.sum(np.abs(final_sku.ypred - final_sku.salesForecast)) / np.sum(final_sku.salesForecast)

print np.sum(np.abs(result3.model_2_ypred - result3.salesForecast)) / np.sum(result3.salesForecast)
print np.sum(np.abs(final_sku_rdc.model_2_ypred - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast)
print np.sum(np.abs(final_sku.model_2_ypred - final_sku.salesForecast)) / np.sum(final_sku.salesForecast)

print np.sum(np.abs(result3.ensemble - result3.salesForecast)) / np.sum(result3.salesForecast)
print np.sum(np.abs(final_sku_rdc.ensemble - final_sku_rdc.salesForecast)) / np.sum(final_sku_rdc.salesForecast)
print np.sum(np.abs(final_sku.ensemble - final_sku.salesForecast)) / np.sum(final_sku.salesForecast)

768821.2473570952
659103.0
628273.9
642050.8492180129
0.6225258683742442
0.3586670960474832
0.2553152699682589
0.6592315955877557
0.3896289505990724
0.2729987668734885
0.5881977519053915
0.30941828048810543
0.2163505769698077
